In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import time
import h5py
import copy
import datetime
import ta
from sklearn.model_selection import train_test_split
# Do you wanna see?
verbose = True

import sys
sys.path.append('../')
#%cd ..

from utils.data_extraction import load_data
from utils.data_cleaning import HFDataCleaning
# from utils.generate_features import candleCreateNP
# from utils.generate_features import candleCreateNP_vect
# from utils.generate_features import candleCreateNP_vect_v2
from utils.generate_features import candleCreateNP_vect_final
# from utils.generate_features import createCandles_test
from utils.generate_features import generateFeatures


#from .utils.data_extraction import load_data
#from .utils.data_cleaning import HFDataCleaning

### Load data etc

In [2]:
# print(os.listdir())
path = 'a:/taqhdf5'  #'a:/taqhdf5'
# allFiles = os.listdir(path)
# print(len(allFiles), allFiles[:5], allFiles[-5:])
# print(allFiles[-10:])

#dates = np.array(['2020040' + str(i) if i < 10 else '202004' + str(i) for i in np.arange(1,32)]).astype(int)
dates = np.array(['20200401','20200402']).astype(int)

# Provide a list of tickers of interest
tickers = ['GOOG']#'MSFT'

# Do we need data on trades, quotes or both?
dataNeeded = 'trades' # 'trades', 'quotes' or 'both'

# run load_data()
if dataNeeded == 'trades':
    tradeData = load_data(dates, tickers, dataNeeded, path, verbose)
elif dataNeeded == 'quotes':
    quoteData = load_data(dates, tickers, dataNeeded, path, verbose)
elif dataNeeded == 'both':
    tradeData, quoteData = load_data(dates, tickers, dataNeeded, path, verbose)

['.ipynb_checkpoints', 'preprocessing.ipynb', 'testing_and_comparing_candlesticks.ipynb']
8515 ['taq_19930315.h5', 'taq_19930104.h5', 'taq_19930317.h5', 'taq_19930105.h5', 'taq_19930316.h5'] ['taqquote_20200522.h5', 'taqquote_20200526.h5', 'taqquote_20200527.h5', 'taqquote_20200528.h5', 'taqquote_20200529.h5']
['taqquote_20200515.h5', 'taqquote_20200518.h5', 'taqquote_20200519.h5', 'taqquote_20200520.h5', 'taqquote_20200521.h5', 'taqquote_20200522.h5', 'taqquote_20200526.h5', 'taqquote_20200527.h5', 'taqquote_20200528.h5', 'taqquote_20200529.h5']
##### Date range #####

Date, Min: 20200401
Date, Max: 20200402

##### Data Extraction begins #####

trade data is being extracted..

### Trade Data ###

The raw H5 trade file contains:  ['TradeIndex', 'Trades'] 

Ticker Information:  (b'GOOG            ', 26900500, 71427) 

Sneak peak of the data

            utcsec ex  cond  volume    price TradeStopStockIndicator corr  \
0  14400048517953  P  @ TI      67  1139.44                           

In [3]:
cleanedData = HFDataCleaning(['P1','p2','t1','p3'],tradeData,'trade',['q'])

### Original

In [4]:
# # Aggregate data in candle sticks
# candleNP = candleCreateNP()

# candleNP.shape

In [5]:
# test_candles=candleCreateNP_vect()
# test_candles

In [6]:
candles = candleCreateNP_vect_final(cleanedData
                         ,1)

# candles_test = createCandles_test(cleanedData
#                                      ,1)

candles

array([[1122.26 , 1127.22 , 1119.51 , 1126.12 ],
       [1127.57 , 1128.   , 1120.   , 1126.92 ],
       [1127.19 , 1127.4  , 1124.02 , 1125.44 ],
       ...,
       [1121.43 , 1122.86 , 1119.76 , 1121.95 ],
       [1122.58 , 1123.6  , 1121.18 , 1122.86 ],
       [1122.725, 1124.   , 1119.64 , 1121.74 ]])

In [7]:
# %timeit candles1 = createCandles(cleanedData,1)
# %timeit candles2 = createCandles_test(cleanedData,1)
# # No difference

# Implementing technical features

A library: https://technical-analysis-library-in-python.readthedocs.io/en/latest/

### Features used in the literature:

* Stochastic K - Implemented
* Stochastic D - Implemented
* Slow Stochastic D - Implemented
* Momentum - Same as difference
* ROC - Implemented
* Williams % R - Implemented
* A/D Oscillator
* Disparity 5 - Implemented
* Disparity 10 - Implemented
* Price Oscillator - (detrended) - Implemented
* Commodity Channel Index - Implemented
* RSI - Impliemented

Formulas: https://papers.ssrn.com/sol3/papers.cfm?abstract_id=876544

* Moving Average - Implemented
* Bias
* Exponential Moving Average - Implemented
* Difference - Same af Momentum
* True Range - (Average) - Implemented
* 

Formulas: https://www.sciencedirect.com/science/article/pii/S0957417407001819?via%3Dihub

#### Other Technical Features
* Moving Average Convergence Divergence (MACD) - Implemented

**Non-classical technical features**

* Bid/Ask prices of top of book
* Spread and mid price based on top og book
* Price derivatives

Formulas: https://www.tandfonline.com/doi/full/10.1080/14697688.2015.1032546?instName=UCL+%28University+College+London%29

In [8]:
features = generateFeatures(candles,['pastobs',
                                      'stok',
                                      'stod',
                                      'sstod',
                                      'wilr',
                                      'roc',
                                      'rsi',
                                      'atr',
                                      'cci',
                                      'dpo',
                                      'sma',
                                      'ema',
                                      'macd','dis5','dis10'],5)

# Labels

In [13]:
returns = ((candles.T[-1][1:]/candles.T[-1][0:-1])-1)*100
# Not tested
classes = 5
labels = np.zeros(returns.shape[0])#-featureWindow
thresholdsMin = [np.array_split(np.sort(returns),classes)[i].min() for i in np.arange(classes)]
thresholdsMax = [np.array_split(np.sort(returns),classes)[i].max() for i in np.arange(classes)]

In [14]:
# Not tested
for i in np.arange(classes):
    
    if i == 0:
        
        labels[(returns <= thresholdsMax[i])] = i
    
    elif i == (classes-1):
        
        labels[(returns >= thresholdsMin[i])] = i
    
    else:
        
        labels[(returns >= thresholdsMin[i])&(returns<=thresholdsMax[i])] = i

In [15]:
labels

array([3., 0., 0., 0., 0., 4., 4., 0., 1., 2., 4., 1., 2., 1., 0., 0., 3.,
       3., 3., 3., 1., 0., 4., 0., 2., 2., 2., 0., 2., 3., 1., 3., 4., 0.,
       4., 4., 4., 4., 2., 1., 3., 3., 4., 2., 2., 3., 1., 1., 0., 3., 4.,
       0., 1., 0., 2., 4., 3., 0., 4., 2., 4., 2., 4., 1., 4., 1., 3., 3.,
       0., 2., 0., 4., 1., 2., 4., 2., 0., 0., 2., 1., 3., 3., 2., 0., 1.,
       1., 4., 1., 4., 0., 3., 0., 3., 4., 3., 2., 2., 2., 3., 2., 2., 1.,
       0., 3., 0., 1., 4., 3., 3., 2., 1., 0., 2., 1., 3., 0., 2., 1., 1.,
       2., 2., 3., 3., 2., 2., 1., 3., 1., 0., 0., 2., 2., 3., 3., 1., 0.,
       4., 1., 3., 3., 1., 3., 2., 2., 3., 2., 4., 1., 0., 2., 0., 1., 2.,
       2., 3., 1., 1., 2., 3., 3., 2., 0., 4., 3., 2., 0., 1., 1., 1., 3.,
       4., 1., 4., 0., 4., 0., 1., 1., 2., 3., 2., 3., 0., 4., 0., 2., 0.,
       0., 1., 1., 2., 1., 2., 3., 4., 2., 3., 0., 1., 4., 4., 3., 2., 2.,
       4., 1., 1., 1., 1., 2., 4., 3., 3., 3., 1., 2., 1., 3., 1., 2., 2.,
       4., 0., 2., 0., 4.

In [17]:
features.shape

(776, 34)

In [18]:
labels.shape

(779,)

In [20]:
features.loc[0:50]

,open_0,high_0,low_0,close_0,open_1,high_1,low_1,close_1,open_2,high_2,...,roc,rsi,atr,cci,dpo,sma,ema,macd,dis5,dis10
0,1122.26,1127.22,1119.51,1126.12,1127.57,1128.000,1120.00,1126.92,1127.19,1127.400,...,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1127.57,1128.00,1120.00,1126.92,1127.19,1127.400,1124.02,1125.44,1125.01,1125.190,...,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1127.19,1127.40,1124.02,1125.44,1125.01,1125.190,1122.97,1123.00,1123.67,1123.680,...,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1125.01,1125.19,1122.97,1123.00,1123.67,1123.680,1120.01,1121.21,1121.27,1121.850,...,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1123.67,1123.68,1120.01,1121.21,1121.27,1121.850,1114.79,1117.44,1117.04,1124.400,...,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,99.704056,NaN
5,1121.27,1121.85,1114.79,1117.44,1117.04,1124.400,1117.04,1124.40,1124.98,1127.990,...,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,99.522445,NaN
6,1117.04,1124.40,1117.04,1124.40,1124.98,1127.990,1124.97,1127.65,1127.63,1128.700,...,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,100.187294,NaN
7,1124.98,1127.99,1124.97,1127.65,1127.63,1128.700,1125.18,1125.51,1126.95,1127.520,...,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,100.437323,NaN
8,1127.63,1128.70,1125.18,1125.51,1126.95,1127.520,1124.27,1124.52,1124.72,1125.890,...,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,100.201916,NaN
9,1126.95,1127.52,1124.27,1124.52,1124.72,1125.890,1123.07,1124.52,1124.12,1126.000,...,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,100.054809,100.026596


## Split data into train and test set

In [16]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.1, random_state=42)

ValueError: Found input variables with inconsistent numbers of samples: [776, 779]